In [5]:
![[ ! -f './input.txt' ]] && wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [6]:
from pathlib import Path


In [7]:
text_path = Path("./input.txt")


In [8]:
text = text_path.read_text()


In [9]:
print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
(text_path.absolute().parent.parent / "project_b").exists()


True

In [11]:
voc = sorted(set(text))


In [12]:
voc_len = len(voc)


In [13]:
voc_len


65

In [46]:
i2s = dict(enumerate(voc))
s2i = {v: k for (k, v) in i2s.items()}


def encode(s: str) -> list[int]:
    return [s2i[k] for k in s]


def decode(i: list[int]) -> str:
    return "".join([i2s[k] for k in i])


a = encode("coco jumbo")
print(a, decode(a))


[41, 53, 41, 53, 1, 48, 59, 51, 40, 53] coco jumbo


In [15]:
import torch


In [16]:
data = torch.tensor(encode(text), dtype=torch.long)


In [17]:
data[:9]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [18]:
data.shape


torch.Size([1115394])

In [19]:
data.dtype


torch.int64

In [20]:
split = int(len(data) * 0.9)


In [21]:
train = data[:split]
val = data[split:]


In [22]:
block_size = 8
train[: block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [69]:
torch.manual_seed(1337)
batch_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_batch(is_train: bool = True):
    data = train if is_train else val
    ids = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ids])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ids])
    x, y = x.to(device), y.to(device)
    return x, y


In [33]:
get_batch()


(tensor([[ 0, 32, 46, 53, 59,  1, 42, 43],
         [46, 43,  1, 56, 43, 57, 58,  1],
         [58, 47, 52, 45,  1, 52, 53,  1],
         [57, 12,  1, 21, 57,  1, 58, 46]]),
 tensor([[32, 46, 53, 59,  1, 42, 43, 57],
         [43,  1, 56, 43, 57, 58,  1, 53],
         [47, 52, 45,  1, 52, 53,  1, 51],
         [12,  1, 21, 57,  1, 58, 46, 47]]))

In [61]:
import torch.nn as nn
from torch.nn import functional as fn
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (batch_size, block_size, vocab_size)

        if targets is None:
            return logits, None
        B, T, C = logits.shape
        logits = logits.reshape(B * T, C)
        targets = targets.reshape(B * T)

        loss = fn.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens=100):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(idx)
            logits = logits[:, -1, :] # (batch_size, vocab_size)
            probs = fn.softmax(logits, dim=-1) # (batch_size, vocab_size)
            idx_next = torch.multinomial(probs, 1) # (batch_size, 1)
            idx = torch.cat([idx, idx_next], dim=1) # (batch_size, T + 1)

        return idx

m = BigramLanguageModel(voc_len)
m = m.to(device)

logits, loss = m(*get_batch())
print(logits.shape, loss)


torch.Size([256, 65]) tensor(4.6485, grad_fn=<NllLossBackward0>)


In [62]:
[decode(i) for i in  m.generate(torch.tensor([[0]]), max_new_tokens=1).tolist()]

['\nS']

In [63]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [64]:
batch_size = 32

for step in range(10000):
    xb, yb = get_batch()
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(f"step {step}: loss {loss.item():.3f}")

step 0: loss 4.656
step 1000: loss 3.724
step 2000: loss 3.162
step 3000: loss 2.912
step 4000: loss 2.654
step 5000: loss 2.620
step 6000: loss 2.489
step 7000: loss 2.531
step 8000: loss 2.547
step 9000: loss 2.543


In [68]:
print(decode(m.generate(torch.tensor([[0]])).tolist()[0]))




Ang
Brd tule t
Coue

e, pakl illferothe, wed
Y:
Bu mmblors, k
n trs brind pis MO:
LARA thcareroous


In [72]:
list(m.parameters())

[Parameter containing:
 tensor([[ 2.3936, -5.1045, -5.3803,  ..., -3.5042, -1.7965, -5.8322],
         [-4.3490, -3.9531, -5.1122,  ..., -6.4487,  0.6181, -3.1308],
         [ 2.6919,  2.2219, -2.9095,  ..., -3.9925, -4.4914, -2.4891],
         ...,
         [-0.9544, -1.0100, -3.7172,  ..., -2.2793, -1.0173, -2.1112],
         [-0.5053,  2.7796, -1.2012,  ..., -3.7570, -4.9620, -4.8453],
         [-3.5804, -1.3296, -4.3497,  ..., -2.7766,  0.2231,  0.2902]],
        requires_grad=True)]